In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 13})
import pickle
from scipy.optimize import curve_fit
from my_functions import schechter, double_schechter, count_true
from minijpas_LF_and_puricomp import effective_volume

In [ ]:
filename = '/home/alberto/cosmos/LAEs/Luminosity_functions/LF_r17-24_z2.5-3.8_ew0_ewoth400_nb/LFs.pkl'
with open(filename, 'rb') as f:
    LFdic = pickle.load(f)

LF_phi = LFdic['LF_total']
LF_bin = LFdic['LF_bins']
LF_yerr_minus = LFdic['LF_total_err'][0]
LF_yerr_plus = LFdic['LF_total_err'][0]
LF_xerr = LFdic['LF_total_err'][0]

In [ ]:
def sch_fit(Lx, Phistar, Lstar):
    return schechter(Lx, Phistar, Lstar, -1.5) * Lx * np.log(10)

In [ ]:
where_fit = (LF_bin > 43.5) & (LF_phi > 0)

In [ ]:
survey_list = [f'minijpasAEGIS00{i}' for i in range(1, 5)] + ['jnep']

hist_mat = [
    np.load(f'npy/hist_i_mat_{survey_name}.npy')
    for survey_name in survey_list
]

In [ ]:
N_iter = len(hist_mat[0])

nb_min = 6
nb_max = 20
bin_width = LF_bin[1] - LF_bin[0]
volume = effective_volume(nb_min, nb_max, 'both')

hist_total = np.sum(hist_mat, axis=0)
LF_total = hist_total / bin_width / volume

popt_list = []
for i in range(N_iter):
    popt, pcov = curve_fit(sch_fit, 10**LF_bin[where_fit], LF_total[i][where_fit],
                        p0=[1e-6, 1e44])
    popt_list.append(popt)

Phi16, Phi50, Phi84 = np.percentile(popt_list, [16, 50, 84], axis=0)

In [ ]:
Lx = np.logspace(42, 47)

popt = Phi50

fig, ax = plt.subplots(figsize=(7, 6))

# Plot the reference LF curves

phistar1 = 3.33e-6
Lstar1 = 44.65
alpha1 = -1.35

phistar2 = -3.45
Lstar2 = 42.93
alpha2 = -1.93

Phi_center = double_schechter(
    Lx, phistar1, 10 ** Lstar1, alpha1, 10 ** phistar2, 10 ** Lstar2, alpha2
) * Lx * np.log(10)

ax.plot(
    np.log10(Lx), Phi_center, ls='-.', alpha=0.7,
    zorder=1, color='C6'
)

phistar1 = 10 ** -3.41
Lstar1 = 10 ** 42.87
alpha1 = -1.7

phistar2 = 10 ** -5.85
Lstar2 = 10 ** 44.6
alpha2 = -1.2

Phi_center = double_schechter(
    Lx, phistar1, Lstar1, alpha1, phistar2, Lstar2, alpha2
) * Lx * np.log(10)

ax.plot(
    np.log10(Lx), Phi_center, ls='-.', alpha=0.7,
    zorder=0, color='C7'
)

# Median fit
fit_list = []
for i in range(N_iter):
    Phi_i = sch_fit(Lx, *tuple(popt))
    fit_list.append(Phi_i)
Phi = np.percentile(fit_list, 50, axis=0)

ax.plot(np.log10(Lx), Phi, label='My fit')

ax.errorbar(LF_bin[where_fit], LF_phi[where_fit],
            yerr=[LF_yerr_minus[where_fit], LF_yerr_plus[where_fit]],
            xerr=LF_xerr[where_fit],
            fmt='s', color='r', capsize=4, label='My points')

ax.set_yscale('log')
ax.set_ylim(1e-8, 5e-3)
ax.set_xlim(42.5, 45.5)
ax.legend()

Phistar = popt[0]
Lstar = np.log10(popt[1])
alpha = -1.5
Phistar_err = (Phi84[0] - Phi16[0]) * 0.5
Lstar_err = (np.log10(Phi84[1]) - np.log10(Phi16[1])) * 0.5

text = fr'''
$\Phi^*=${Phistar:0.1e}$\pm${Phistar_err:0.1e}
$L^*=${Lstar:0.2f}$\pm${Lstar_err:0.2f}
$\alpha=${alpha:0.2f} (fixed)
'''
ax.text(42.7, 1e-7, text)

plt.show()